<a href="https://colab.research.google.com/github/DuplamenteH/Ds_estudos/blob/main/pyspark_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## instalação do pyspark no colab


para instalar o pyspark primeiro teremos que instalar  -> java , spark
configurar java_home, e spark_home, instalar o findspark

instalação de configuração das variaveis de ambiente

In [ ]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
 
# tornar o pyspark "importável"
#import findspark
findspark.init('spark-3.0.1-bin-hadoop2.7')

## importação

Depois de importado e configurado vamos dar inicio ao spark

importaremos sparkSession para inicio a nossa sessão

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("Data_Wrangling").getOrCreate()

In [ ]:
!wget -q https://raw.githubusercontent.com/Apress/applied-data-science-using-pyspark/main/Ch02/Chapter2_Data/movie_data_part1.csv

In [ ]:
dados = 'movie_data_part1.csv'
file_type = "csv"
infer_schema = "False"
first_row_is_header = "True"
delimiter  = "|"

primeira forma de carregar os dados

In [ ]:
df = spark.read.format(file_type)\
.option("inferSchema",infer_schema)\
.option("header",first_row_is_header)\
.option("sep",delimiter)\
.load(dados)

    

## Lendo os dados de uma tabela hive


df2 = spark.sql("select * from database.table_name")

printamos os schema, assim podemos ver as colunas, se existem valores faltantes, e o tipo de cada coluna

In [ ]:
df.printSchema()

com a função df.dtypes, retornamos uma lista de tuplas com o nome da coluna e o tipo da coluna apenas

In [ ]:
df.dtypes

agr vamos saber o tamanho desse dataset, quantas linha tem o mesmo

In [ ]:
df.count()

uma maneira mai bonita seria assim:

In [ ]:
print("O total dos nossos registros é de  : {}".format(df.count()))

Coletando subcolunas para ter uma visão geral dos dados

In [ ]:
select_columns = ['id', 'budget', 'popularity', 'release_date',
                  'revenue', 'title']

In [ ]:
df =df.select(*select_columns)
df.show(30,False)

## Valores faltantes do nosso dataset

algumas maneiras de calcular os valores faltantes

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df.filter((df['popularity']=='')|df['popularity'].isNull()|isnan(df['popularity'])).count()

na operação usando o condicional OR -> |,

na primeira condição achamos as strings nulas presentes em popularity comparando com strings vazias, na segunda condição chamamos a função .isNull() para a colunar popularity que retorna true quando alguma linha popularity é verdadeira e false se o contrario acontecer, na terceira condição usando a função isnan()-> not a number.

A segunda forma de achar a quantidade de valores faltantes é da seguinte forma.

In [ ]:
df.select([count(when((col(c)=='')| col(c).isNull()|isnan(c),c)).
           alias(c) for c in df.columns]).show()

One-Way-Frequencies

Agr vamos calcular as frequencias do nosso dataset, vamos verificar abaixo os titulos refletidos

In [ ]:
df.groupBy(df['title']).count().show()

Bom agr vamos ordenar nossas frenquencia para exibir as 10 maiores.

In [ ]:
df.groupby(df['title']).count().sort(desc("count")).show(10,False)

ordenando e filtrando nossas one-way frequencies

1º. Vamos criar um dataset temporario para eliminar os valores faltantes

In [ ]:
df_temp = df.filter((df['title']!='')&(df['title'].isNotNull())&(~isnan(df['title'])))

verificando nosso datafreme temporario

In [ ]:
df_temp.groupby(df_temp['title']).count().filter("`count`>4").sort(col("count").desc()).show(20,False)

Vemos na tabela acima que os dados faltantes foram eliminados.

No código abaixo vamos mostrar os titulos que são repetidos mais de 4 vezes

In [ ]:
df_temp.groupby(df_temp['title']).count().filter("`count`>=4").sort(col("count").desc()).count()

Agr vamos deletar o dataframe temporario

In [ ]:
del df_temp

Casting Variables

Antes do casting

In [ ]:
df.dtypes

In [ ]:
df = df.withColumn('budget',df['budget'].cast("float"))

depois do casting

In [ ]:
df.dtypes

Agora vamos usar outro método para fazer o casting, vamos fazer um loop pela coluna que queremos mudar seu tipo e ai sim fazer a mudança

In [ ]:
from pyspark.sql.types import *

selecionando as colunas

In [ ]:
int_var = ['id']
float_vars = ['budget', 'popularity', 'revenue']
date_vars = ['release_date']

fazendo a conversão

In [ ]:
for column in int_var:
    df = df.withColumn(column,df[column].cast(IntegerType()))

for column in float_vars:
    df = df.withColumn(column,df[column].cast(FloatType()))

for column in date_vars:
    df = df.withColumn(column,df[column].cast(DateType()))


df.dtypes

mostrando o dataset

In [ ]:
df.show(20,False)

## Estatisticas Descritivas

In [ ]:
df.describe().show()

Retirando os valores 0 do budget

In [ ]:
df_temp = df.filter((df['budget']!=0)&(df['budget'].isNotNull())&(~isnan(df['budget'])))

verificando se deu certo nossa operação de remover os falores faltantes do nosso df

In [ ]:
df_temp.describe().show()

calculando e ajustando a media


In [ ]:
median = df_temp.approxQuantile('budget',[0.5],0.1)

print('A media do budget é '+str(median))

Valores unicos/distintos e counts

contando os titulos distintos

In [ ]:
df.agg(countDistinct(col('title')).alias("count")).show()

Contando os titulos distintos

In [ ]:
df.select('title').distinct().show(20,False)

Bom, seria uma boa identificar os titulos distintos por ano de lançamento

vamos extrair o ano da nossa coluna realease date

In [ ]:
df_date = df.withColumn('release_year', year('release_date'))

com isso tbm conseguimos extrair os meses e os dias

In [ ]:
df_date = df_date.withColumn('release_month', month('release_date'))
df_date = df_date.withColumn('release_day', dayofmonth('release_date'))

calculando os dados distintos por ano


In [ ]:
df_date.groupBy("release_year").agg(countDistinct("title")).show(40,False)

## Filtering


semelhante ao metodo **where** , o metodo filter serve para realizar buscas dentro da nossa base de dados.

In [ ]:
df.filter(df['title'].like('Meet%')).show(10,False)

In [ ]:
#titulos que não terminam com a letra "S"

df.filter(~df['title'].like('%s')).show(10,False)

In [ ]:
#filmes onde a contem a palavra over
df.filter(df['title'].rlike('\w*ove')).show(10,False)

In [ ]:
#outro modo de escrever
df.filter(df.title.contains('over')).show()

In [ ]:
# identificando os filmes que iniciam com "re"
df.select(df.colRegex("`re\w*`")).printSchema()


In [ ]:
df.select(df.colRegex("`\w*e`")).printSchema()

O que é regex?
<br/> De maneira geral usamos regex para identicar todos os carecteres e numeros de um "texto".

* \d -> Identifica numeros entre 0 e 9
* \w -> Identifica todos o caractres maiusculos e minusculos do alfabeto e numero de 0 a 9 [A-Z, a-z, ,0-9] 
* \s - Espaços em branco
* . Todo caractere

também podemos usar isso de maneira quantitativa:
*   *_=> 0 ou mais caracteres
*   +_=> 1 ou mais caracteres
* {min,max} - especificar o tamanho dos caracteres
* {n} - numero exato dos n caracteres


## Criando novas colunas


Cria novas features (colunas) é algo que acontece em qualquer projeto de data science, o spark oferece diversos meios para isso acontecer, um metodo simples seria usar o "withColumn". Vamos calcular a variancia da popularidade dos nossos filmes, variancia indica o quanto os numeros estão distantes da media.

$ s^2 = Σ(X_i - \bar{X})^2 / (n-1)$

In [ ]:
mean_pop = df.agg({'popularity': 'mean'}).collect()[0]['avg(popularity)']
count_obs = df.count()

In [ ]:
df = df.withColumn('mean_popularity', lit(mean_pop))

In [ ]:
df = df.withColumn('variance',pow((df['popularity'] - df['mean_popularity']),2))

In [ ]:
variance_sum = df.agg({'variance':'sum'}).collect()[0]['sum(variance)']

variance_population = variance_sum/(count_obs-1)



In [ ]:
print("Variancia = {}".format(variance_population))

In [ ]:
def new_cols(budget, popularity):
    if budget < 10000000: budget_cat = 'Small'
    elif budget < 100000000: budget_cat = 'Medium'
    else: budget_cat = 'Big'
    if popularity < 3: ratings = 'Low'
    elif popularity < 5: ratings = 'Mid'
    else: ratings = 'High'
    return budget_cat, ratings

In [ ]:
# Pesquisar melhor o metodo  StructType
ufdB = udf(new_cols, StructType([StructField("budget_cat",StringType(),True), StructField("ratings",StringType(),True)]))

temp_df = df.select('id','budget','popularity').withColumn("newcat",ufdB("budget","popularity"))

df_with_newcols = temp_df.select('id','budget','popularity','newcat')\
.withColumn('budget_cat',temp_df.newcat.getItem('budget_cat'))\
.withColumn('ratings',temp_df.newcat.getItem('ratings'))\
.drop('newcat')


In [ ]:
df_with_newcols.show(15,False)

# Fazer o exercicio da pagina 80